## Cleaning US Census Data using Python
From my Codecademy course on data cleaning with Python

Robert Hall 12.20.2023

In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl as opx
from openpyxl.utils.dataframe import dataframe_to_rows as dtr
import glob

In [24]:
# load all 'states*' files into one series

files = glob.glob('states*.csv')

In [25]:
# convert data from all 10 files into
# dataframes, then aggregate all dataframes
# to one 'us_census' dataframe

states_list = []
for f in files:
  data = pd.read_csv(f)
  states_list.append(data)
us_census = pd.concat(states_list)

In [26]:
# print first 5 rows of us_census dataframe
# to verify the aggregation worked

print(us_census.head())

   Unnamed: 0       State  TotalPop             Hispanic                White  \
0           0     Alabama   4830620  3.7516156462584975%     61.878656462585%   
1           1      Alaska    733375   5.909580838323351%  60.910179640718574%   
2           2     Arizona   6641928  29.565921052631502%  57.120000000000026%   
3           3    Arkansas   2958208   6.215474452554738%   71.13781021897813%   
4           4  California  38421464  37.291874687968054%   40.21578881677474%   

                 Black                Native                Asian  \
0   31.25297619047618%   0.4532312925170065%  1.0502551020408146%   
1  2.8485029940119775%    16.39101796407186%   5.450299401197604%   
2  3.8509868421052658%     4.35506578947368%   2.876578947368419%   
3  18.968759124087573%   0.5229197080291965%  1.1423357664233578%   
4   5.677396405391911%  0.40529206190713685%  13.052234148776776%   

                Pacific              Income            GenderPop  
0  0.03435374149659865%  $43296

In [27]:
# print datatypes to verify which ones are objects and
# which need to be converted

print(us_census.dtypes)

Unnamed: 0     int64
State         object
TotalPop       int64
Hispanic      object
White         object
Black         object
Native        object
Asian         object
Pacific       object
Income        object
GenderPop     object
dtype: object


In [28]:
# convert income datatype from object to numeric
# and splicing out the '$' character from before
# each value to make the data calculable.

us_census.Income = us_census['Income'].replace('[\$,]', '', regex=True)
us_census.Income = pd.to_numeric(us_census.Income)

In [29]:
# split 'GenderPop' into two different columns -- men and 
# women -- which are populations of each gender of each 
# US state instance.

# take out the 'M' and 'F' denotations between the male and female
# populations
us_census.GenderPop = us_census.GenderPop.str.replace('[M,F]', '', regex=True)

# print first 5 rows of GenderPop to make sure it went accordingly
print(us_census.GenderPop.head())

0      2341093_2489527
1        384160_349215
2      3299088_3342840
3      1451913_1506295
4    19087135_19334329
Name: GenderPop, dtype: object


In [30]:
# split each instance of GenderPop between the '_' character dividing
# male and female populations, into lists of [0=male, 1=female]
gender_split = us_census.GenderPop.str.split('_')

# print first 5 rows of gender_split to make sure it went accordingly
print(gender_split.head()) 

0      [2341093, 2489527]
1        [384160, 349215]
2      [3299088, 3342840]
3      [1451913, 1506295]
4    [19087135, 19334329]
Name: GenderPop, dtype: object


In [31]:
# parse each instance of GenderSplit into new columns "Men" with male
# populations (where gender_split[0]) and "Women" with female populations
# (where gender_split[1])
us_census["Men"] = gender_split.str.get(0)
us_census["Women"] = gender_split.str.get(1)

In [32]:
# convert these columns to numeric datatypes from objects
us_census.Men = pd.to_numeric(us_census.Men)
us_census.Women = pd.to_numeric(us_census.Women)

In [33]:
# print first 5 rows of "Men" column to verify that all went
# smoothly

print(us_census.Men.head())

0     2341093
1      384160
2     3299088
3     1451913
4    19087135
Name: Men, dtype: int64


In [34]:
# print first 5 rows of "Women" column to verify that all went
# smoothly


print(us_census.Women.head())

0     2489527.0
1      349215.0
2     3342840.0
3     1506295.0
4    19334329.0
Name: Women, dtype: float64


In [36]:
# drop the now- redundant and useless GenderPop column

us_census = us_census.drop('GenderPop', axis=1)

In [37]:
# fill in null (missing) values in the "Women" column with
# the differences between the total population and the male
# population of each state

us_census.Women = us_census.Women.fillna(value=(us_census.TotalPop - us_census.Men))

In [44]:
# drop duplicate rows, where there are multiple rows per
# one state

us_census = us_census.drop_duplicates(subset=['State'])

In [49]:
# write cleaned and updated US census data to
# new csv file

us_census.to_csv('states.csv', sep=',')